In [1]:
import EXOSIMS as EX, os.path
import EXOSIMS.MissionSim as msim
import EXOSIMS.StarCatalog.EXOCAT1 as exc
import numpy as np
import math as ma
import sys, pdb
import matplotlib.pyplot as plt
import astropy.units as u
import pickle, json, warnings, astropy
%matplotlib

Using matplotlib backend: TkAgg


###  Use script files saved in EXOSIMS path

In [2]:
# LOAD JSON SCRIPT FILE
jfile = 'sampleScript_rpatel.json'
scriptfile = os.path.join(os.path.abspath(''),'scripts',jfile)
print scriptfile

/Users/rpatel/Dropbox (Personal)/Research/WFIRST/EXOSIMSTesting/scripts/sampleScript_rpatel.json


In [5]:
script = open(scriptfile).read()
specs_from_file = json.loads(script)

In [6]:
# QUESTION -- DO I HAVE TO RUN THIS EACH TIME OR IS THERE A WAY TO SAVE/LOAD THE OUTPUT?
%time sim = msim.MissionSim(scriptfile)

MissionSim.pyc: Beginning logging to "logfile.txt" at level INFO
MissionSim: Seed is:  179444526
Imported SurveyEnsemble (prototype module) from EXOSIMS.Prototypes.SurveyEnsemble
Imported KnownRVPlanetsUniverse (specific module) from EXOSIMS.Simulat[...]se.KnownRVPlanetsUniverse
Imported KnownRVPlanetsTargetList (specific module) from EXOSIMS.TargetList.KnownRVPlanetsTargetList
Imported StarCatalog (prototype module) from EXOSIMS.Prototypes.StarCatalog
Imported Nemati (specific module) from EXOSIMS.OpticalSystem.Nemati
Imported Stark (specific module) from EXOSIMS.ZodiacalLight.Stark
Imported PostProcessing (prototype module) from EXOSIMS.Prototypes.PostProcessing
Imported GalaxiesFaintStars (specific module) from EXOSIMS.BackgroundSources.GalaxiesFaintStars
Imported GarrettCompleteness (specific module) from EXOSIMS.Completeness.GarrettCompleteness
Imported KnownRVPlanets (specific module) from EXOSIMS.PlanetPopulation.KnownRVPlanets
Imported Forecaster (specific module) from EXOSIMS.

In [27]:
sim.SurveySimulation.run_sim()

Simulation finishing OK. Results stored in SurveySimulation.DRM


'Simulation finishing OK. Results stored in SurveySimulation.DRM'

## Module Objects

In [3]:
halo = pickle.load( open(u'/Users/rpatel/Dropbox (Personal)/Research/WFIRST/EXOSIMS/EXOSIMS/Observatory/L2_halo_orbit_six_month_v2.p', "rb" ) )

In [8]:
import scipy.interpolate as interpolate
import astropy.units as u
import pickle
orbit_time = halo['t'].flatten()/(2*np.pi)*u.year
orbit_pos = halo['state'][:,0:3]* u.AU
orbit_vel = halo['state'][:,3:6]*u.AU/u.year*(2*np.pi)
orbit_interp = interpolate.interp1d(orbit_time.value, orbit_pos.value.T,kind='cubic')
orbit_V_interp = interpolate.interp1d(orbit_time.value, orbit_vel.value.T,kind='cubic')


#simresults = 'simresults_%iyrs_%.0Estars_%s.pickle' %(mlife,nsimplanets,jfilebase)
datfile = 'orbit_test.pickle'

data = {'pos':orbit_pos.value.T,'time':orbit_time.value,'vel':orbit_vel.value.T}


handler = open(datfile,'wb')
pickle.dump(data,handler)
handler.close()

In [84]:
# Stored detection information -- if any -- in DRM Dictionary 
DRM = sim.SurveySimulation.DRM

In [85]:
DRM

[]

In [49]:
# Simulation specifications ; i.e., all parameters used in simulation
AllSpecs = sim.genOutSpec()

In [51]:
WA = sim.SimulatedUniverse.WA

In [53]:
iwa = AllSpecs['IWA'] * 1000

# Right now, I'm loading obvious objects into local variables
# We can add more depending on what we need for the interface

### modules:

In [9]:
TL = sim.TargetList
SC = sim.StarCatalog
SU = sim.SimulatedUniverse
SSim = sim.SurveySimulation
OS = sim.OpticalSystem
ZL = sim.ZodiacalLight
BS = sim.BackgroundSources
CP = sim.Completeness
PP = sim.PlanetPopulation
PM = sim.PlanetPhysicalModel

## Stellar Properties

In [10]:
Name   = TL.Name
Spec   = TL.Spec
parx   = TL.parx
Umag   = TL.Umag
Bmag   = TL.Bmag
Vmag   = TL.Vmag
Rmag   = TL.Rmag
Imag   = TL.Imag
Jmag   = TL.Jmag
Hmag   = TL.Hmag
Kmag   = TL.Kmag
dist   = TL.dist
BV = TL.BV
MV = TL.MV
BC = TL.BC
L  = TL.L
coords = TL.coords
pmra   = TL.pmra
pmdec  = TL.pmdec
rv = TL.rv
Binary_Cut = TL.Binary_Cut
#maxintTime = OS.maxintTime
comp0  = TL.comp0
MsEst  = TL.MsEst
MsTrue = TL.MsTrue
nStars = TL.nStars

star_prop = {'Name':Name,'Spec':Spec,'parx':parx,'Umag':Umag,'Bmag':Bmag,'Vmag':Vmag,
            'Imag':Imag,'Jmag':Jmag,'Hmag':Hmag,'Kmag':Kmag,'dist':dist,'BV':BV,
            'MV':MV,'Lum':L,'coords':coords,'pmra':pmra,'pmdec':pmdec,'rv':rv,
            'Binary_Cut':Binary_Cut,'comp0':comp0,'MsEst':MsEst,
            'MsTrue':MsTrue,'nStars':nStars}#,'maxintTime':maxintTime}


## Planets Generated Properties

In [11]:
try:
    arange = TL.arange 
    erange = TL.erange
    wrange = TL.wrange
    Orange = TL.Orange
    prange = TL.prange
    Irange = TL.Irange
    Rrange, Mprange = TL.Rrange, TL.Mprange
    rrange = TL.rrange
    synplanet_prop = {'arange':arange,'erange':erange,'wrange':wrange,'Orange':Orange,
                      'prange':prange,'Irange':Irange,'Rrange':Rrange,'Mprange':Mprange,
                      'rrange':rrange}
except AttributeError:
    print 'Sorry.. no go.'
    print 'Not simulated planets.'
    synplanet_prop = None

try:
    nPlans, plan2star = SU.nPlans, SU.plan2star
    sInds = SU.sInds
    # ORBTIAL PARAMETERS
    sma,e,w,O,I = SU.a, SU.e, SU.w, SU.O, SU.I
    # PLANET PROPERTIES
    Mp,Rp = SU.Mp, SU.Rp
    # POSITION AND VELOCITY VECTOR OF PLANET
    r, v = SU.r, SU.v
    # ALBEDO
    p = SU.p
    fEZ = SU.fEZ

    empplanet_prop = {'nplans':nPlans,'plan2star':plan2star,'sInds':sInds,'sma':sma,
                      'e':e,'w':w,'O':O,'I':I,'Mp':Mp,'Rp':Rp,'r':r,'v':v,'p':p,'fEZ':fEZ}
except AttributeError:
    print 'Sorry.. no go.'
    print 'Not ``real`` planets.'
    empplanet_prop = None
    

Sorry.. no go.
Not simulated planets.


## Additional simulation results -- add more as needed

# number of visits to each star -- double check
nvisits_perstar = SU.Completeness.

etc_data = {'nvisits':nvisits_perstar}

## Dump Survey Simulation Results -- not completeness

In [12]:
mlife = AllSpecs['missionLife']
#nsimplanets = specs_from_file['Nplanets']
jfilebase = jfile.strip('.json').strip('template_')

#simresults = 'simresults_%iyrs_%.0Estars_%s.pickle' %(mlife,nsimplanets,jfilebase)
simresults = 'simresults_%.2fyrs_%s.pickle' %(mlife,jfilebase)
simfile = os.path.join('/Users/rpatel/Dropbox/Research/WFIRST/EXOSIMSTesting/SimResults/',simresults)

data = {'DRM':DRM,'empplanet_prop':empplanet_prop,'synplanet_prop':synplanet_prop,
        'star_prop':star_prop,'AllSpecs':AllSpecs}#,'etc_data':etc_data}

handler = open(simfile,'wb')
pickle.dump(data,handler)
handler.close()

In [13]:
simresults

'simresults_2.00yrs_rpateltest_KnownRV_2year.pickle'